In [1]:
from sodapy import Socrata
import json
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from config import apiKey

from urllib.request import urlopen
from scipy import stats
import gmaps
import plotly.express as px



# Use open client to obtain data
client = Socrata("data.cdc.gov", apiKey)
results = client.get("dnhi-s2bf", limit = 1000000)

# Convert to pandas DataFrame
death_quarterly = pd.DataFrame.from_records(results)
death_quarterly.head()


ModuleNotFoundError: No module named 'config'

In [ ]:
death_quarterly_1 = death_quarterly.drop(columns=['footnote'])
death_quarterly_2 = death_quarterly_1.dropna(subset =['covid_19_deaths','total_deaths'])


In [ ]:
death_quarterly_2['total_deaths'] = death_quarterly_2['total_deaths'].astype(int)
death_quarterly_2['covid_19_deaths'] = death_quarterly_2['covid_19_deaths'].astype(int)
death_gb = death_quarterly_2.groupby(['fips_code'])
death_gb_df = death_gb.sum()

In [ ]:
death_gb_df['covid_portion_of_total_deaths'] = death_gb_df['covid_19_deaths']/death_gb_df['total_deaths']
death_gb_df = pd.DataFrame(death_gb_df)
death_gb_df.reset_index(inplace=True)


In [ ]:
death_gb_df

In [ ]:
death_gb_df.reset_index(inplace=True)

In [ ]:
death_gb_df.describe(include='all')

In [ ]:
from urllib.request import urlopen
import json
import requests
from scipy import stats
import pandas as pd
import numpy as np
import gmaps
from scipy.stats import linregress
from matplotlib import pyplot as plt
import plotly.express as px


url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
response = requests.get(url).json()
response

In [ ]:
fig = px.choropleth(death_gb_df, geojson=response, locations='fips_code', color='covid_portion_of_total_deaths',
                           color_continuous_scale="blues",
                           range_color=(0, .35),
                           scope="usa",
                           labels={'covid deaths as % of total deaths':'covid_portion_of_total_deaths'}
                          )
fig.show()

In [ ]:
death_gb_df.describe()
death_df = death_gb_df.rename(columns = {'fips_code':'fips'})
death_df.head()

In [ ]:
# Use open client to obtain data
client = Socrata("data.cdc.gov", apiKey)
results = client.get("8xkx-amqh", limit = 2000000)
                     
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df.head()

In [ ]:
results_df_columns = results_df[['date','fips','recip_state','recip_county',
                                     'completeness_pct','administered_dose1_recip',
                                     'administered_dose1_pop_pct','series_complete_yes',
                                     'series_complete_pop_pct','metro_status','census2019']]

In [ ]:
clean_df = results_df_columns.dropna()
clean_df['date'] = pd.to_datetime(clean_df['date']).dt.date

In [ ]:
clean_df['month'] = pd.DatetimeIndex(clean_df['date']).month

In [ ]:
clean_df['quarter'] = pd.DatetimeIndex(clean_df['date']).quarter
clean_df['year'] = pd.DatetimeIndex(clean_df['date']).year
clean_df.head()

In [ ]:
grouped_fips = clean_df.groupby(['fips'])
max_group = grouped_fips.max()

In [ ]:
max_dose = pd.DataFrame(max_group[['administered_dose1_recip','administered_dose1_pop_pct']]) 
max_dose.head()

In [ ]:
merged_df = death_df.merge(max_dose,on='fips')
merged_df.head()

In [ ]:
x_axis = merged_df['administered_dose1_pop_pct']
y_axis = merged_df['covid_portion_of_total_deaths']

plt.scatter(x_axis,y_axis)
plt.show()

In [ ]:
death_quarterly_2 = death_quarterly.rename(columns = {'fips_code' : 'fips'})
death_quarterly_2['year']= death_quarterly_2['year'].astype(int)
death_quarterly_2['quarter']=death_quarterly_2['quarter'].astype(int)
clean_df['year']=clean_df['year'].astype(int)
clean_df['quarter']=clean_df['quarter'].astype(int)

In [ ]:
len(clean_df)

In [ ]:
total_quarter = pd.merge(death_quarterly_2,clean_df, on = ('fips','year','quarter'),how ='inner')
total_quarter.head()
total_quarter_2 = total_quarter.loc[(total_quarter['state_of_residence']=='Florida')|(total_quarter['state_of_residence'] =='Texas')|(total_quarter['state_of_residence'] =='California'),:]
total_quarter_2.head()
len(total_quarter)

In [ ]:
total_quarter_2['quarter_number'] = pd.to_datetime(total_quarter_2['year'].astype(str) + 'Q' + total_quarter_2['quarter'].astype(str))
url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"

response = requests.get(url).json()
response

In [ ]:
total_quarter_2.head(25)

In [ ]:
# x_axis= total_quarter_2['quarter_number']
# y_axis= total_quarter_2['']


In [ ]:
total_quarter_2.dropna(subset = ['covid_19_deaths','total_deaths'])
total_quarter_2['covid_death_rate']= total_quarter_2['covid_19_deaths'].astype(float)/total_quarter_2['total_deaths'].astype(float)
total_quarter_2.head(25)


In [ ]:
len(total_quarter_2)

In [ ]:
total_quarter_2.drop_duplicates(subset =['quarter_number','fips'],inplace=True)
                                                                                                                                       

In [ ]:
total_quarter_2.head()

In [ ]:
len(total_quarter_2)
# fig = px.choropleth(death_gb_df, geojson=response, locations='fips_code', color='covid_portion_of_total_deaths',
#                            color_continuous_scale="bluered",
#                            range_color=(0.1, .25),
#                            scope="usa",
#                            labels={'COVID as % of Total Deaths':'covid_portion_of_total_deaths'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [ ]:
# x_axis= total_quarter_2['quarter_number']
# y_axis= total_quarter_2['']


In [ ]:
total_quarter_2.columns

In [ ]:
total_quarter_2['administered_dose1_recip'] = total_quarter_2['administered_dose1_recip'].astype(float) 
total_quarter_2['census2019'] = total_quarter_2['census2019'].astype(float) 
total_quarter_2['covid_19_deaths'] = total_quarter_2['covid_19_deaths'].astype(float) 
total_quarter_2['total_deaths'] = total_quarter_2['total_deaths'].astype(float) 



In [ ]:
california_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'California',:]
california_df.head()

cali_quarter_group = california_df.groupby('quarter_number')
cali_quarter_df = cali_quarter_group.sum()


In [ ]:
cali_quarter_df.head()

In [ ]:
cali_quarter_df['covid_death_rate']= cali_quarter_df['covid_19_deaths']/cali_quarter_df['total_deaths']
cali_quarter_df['vaccine_rate']= cali_quarter_df['administered_dose1_recip']/cali_quarter_df['census2019']

cali_quarter_df.head()

In [ ]:
x_axis = cali_quarter_df.index
y_axis = cali_quarter_df['vaccine_rate']

plt.plot(x_axis, y_axis)
plt.title("Title")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()


In [ ]:
total_quarter['administered_dose1_recip'] = total_quarter['administered_dose1_recip'].astype(float) 
total_quarter['census2019'] = total_quarter['census2019'].astype(float) 
total_quarter['covid_19_deaths'] = total_quarter['covid_19_deaths'].astype(float) 
total_quarter['total_deaths'] = total_quarter['total_deaths'].astype(float) 



In [ ]:
florida_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'Florida',:]
florida_df.head()

FL_quarter_group = florida_df.groupby('quarter_number')
FL_quarter_df = FL_quarter_group.sum()

In [ ]:
FL_quarter_df['covid_death_rate']= FL_quarter_df['covid_19_deaths']/FL_quarter_df['total_deaths']
FL_quarter_df['vaccine_rate']= FL_quarter_df['administered_dose1_recip']/FL_quarter_df['census2019']

FL_quarter_df.head()

In [ ]:
x2_axis = FL_quarter_df.index
y2_axis = FL_quarter_df['vaccine_rate']

plt.plot(x2_axis, y2_axis)
plt.title("Title")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

In [ ]:
texas_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'Texas',:]
texas_df.head()

TX_quarter_group = texas_df.groupby('quarter_number')
TX_quarter_df = TX_quarter_group.sum()
TX_quarter_df.head()

In [ ]:
TX_quarter_df['covid_death_rate']= TX_quarter_df['covid_19_deaths']/TX_quarter_df['total_deaths']
TX_quarter_df['vaccine_rate']= TX_quarter_df['administered_dose1_recip']/TX_quarter_df['census2019']

TX_quarter_df.head()

In [ ]:
x3_axis = TX_quarter_df.index
y3_axis = TX_quarter_df['covid_19_deaths']/TX_quarter_df['total_deaths']

plt.plot(x3_axis, y3_axis)
plt.title("Title")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

In [ ]:
(TX_quarter_df['covid_death_rate']*100).sort_index().plot.line()